# Loading packages and objects

**To Activate and Use Your JupyterLab venv:**

`source ~/venv-jlab/bin/activate`

`jupyter lab`


**This sets up your shell to use the Python and all installed packages from `~/venv-jlab`**

**Once you’re done, you can exit the venv with:**

`deactivate`

In [ ]:
source("/home/ridvan/scRIPT/settings.R")
source("/home/ridvan/scRIPT/utils.R")

In [ ]:
BPPARAM <- BiocParallel::bpparam()
BPPARAM$workers = 20

# Multi core using future - built in to seurat
plan("multicore", workers = 20)
options(future.globals.maxSize = 200 * 1024 ^ 3) # for 200 Gb RAM

getwd()

In [ ]:
obj <- readRDS('../../cmo_version3.rds')
obj

In [ ]:
io$output_rds_dir
atlas <- readRDS(paste0(io$output_rds_dir, 'extended_atlas_cellranger_features_seurat.rds'))
atlas

# Minor Adjustments

In [ ]:
table(obj$v7k75pca_stage_predicted.id)

In [ ]:
# Specify the desired order of the levels
stage_levels <- c(
  "E6.5", "E7.0", "E7.25", "E7.5", "E7.75", "E8.0", "E8.25", "E8.5",
  "E8.75", "E9.0", "E9.25", "E9.5", "Mixed gastrulation"
)

# Create a new factor column in your meta.data with the specified order
obj@meta.data$v7k75pca_stage_predicted.id_num <-
  factor(obj$v7k75pca_stage_predicted.id, levels = stage_levels)

# Check that the levels are correct
table(obj@meta.data$v7k75pca_stage_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_stages <- obj@meta.data$v7k75pca_stage_predicted.id_num

In [ ]:
obj$lv_stages_score <- obj@meta.data$v7k75pca_stage_prediction.score.max

In [ ]:
obj$lv_stages_ch <- as.character(obj$lv_stages) 

In [ ]:
table(obj$v7k75pca_anatomy_predicted.id)

In [ ]:
# Specify the desired order (spelling corrected and capitalized)
anatomy_levels <- c(
  "Pooled", 
  "YS", 
  "EP", 
  "Anterior", 
  "Posterior", 
  "Anterior section", 
  "Medial section", 
  "Posterior section"
)

# Create new factor column with ordered levels
obj@meta.data$v7k75pca_anatomy_predicted.id_num <- factor(
  obj$v7k75pca_anatomy_predicted.id,
  levels = anatomy_levels
)

# Check the result
table(obj@meta.data$v7k75pca_anatomy_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_anatomy <- obj@meta.data$v7k75pca_anatomy_predicted.id_num

In [ ]:
obj$lv_anatomy_score <- obj@meta.data$v7k75pca_anatomy_prediction.score.max

In [ ]:
obj$lv_anatomy_ch <- as.character(obj$lv_anatomy) 

In [ ]:
table(obj$v7k75pca_somite_count_predicted.id)

In [ ]:
# Specify the desired level order (numbers as characters, pooled last)
somite_levels <- c(
  "6", "7", "8", "10", "12", "14", "15", "16", "18", 
  "20", "21", "22", "23", "24", "Pooled"
)

# Create new factor column with ordered levels
obj@meta.data$v7k75pca_somite_count_predicted.id_num <- factor(
  obj$v7k75pca_somite_count_predicted.id,
  levels = somite_levels
)

# Check the result
table(obj@meta.data$v7k75pca_somite_count_predicted.id_num, useNA = "always")


In [ ]:
obj$lv_somite <- obj@meta.data$v7k75pca_somite_count_predicted.id_num

In [ ]:
obj$lv_somite_score <- obj@meta.data$v7k75pca_somite_count_prediction.score.max

In [ ]:
obj$lv_somite_ch <- as.character(obj$lv_somite) 

In [ ]:
obj$lv_cell_type_score <- obj@meta.data$v7k_pca75_prediction.score.max

In [ ]:
obj$lv_cell_type <- obj@meta.data$v7k_pca75_predicted.id

In [ ]:
obj$lv_cell_to_cell <- obj@meta.data$v7k75pca_cell_predicted.id
obj$lv_cell_to_cell_score <- obj@meta.data$v7k75pca_cell_prediction.score.max

# Confidence Score

## Embryonic Stages

In [ ]:
length(unique(obj@meta.data$lv_stages_score))

round(length(unique(obj@meta.data$lv_stages_score))/100, 0)

In [ ]:
# Calculate 75th percentile
percentile_75 <- quantile(obj@meta.data$lv_stages_score, probs = 0.75, na.rm = TRUE)

# Compute max bin count for label positioning
hist_data <- hist(obj@meta.data$lv_stages_score, breaks = 1000, plot = FALSE)
max_y <- max(hist_data$counts)

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(data = obj@meta.data, aes(x = lv_stages_score)) +
  geom_histogram(
    bins = 1000, 
    color = 'blue', 
    fill = 'steelblue', 
    alpha = 0.85
  ) +
  geom_vline(xintercept = percentile_75, linetype = "dashed", color = "red", size = 0.5) +
  annotate(
    "text", 
    x = percentile_75, 
    y = max_y * 0.95, 
    label = "75th percentile", 
    size = 4.5, 
    color = "red", 
    fontface = "bold",
    hjust = -0.1
  ) +
  labs(
    title = "Embryonic Stages",
    x = "Per-Cell Assignment Confidence Score (0 = low, 1 = high)",
    y = "Number of Cells"
  ) +
  theme_classic(base_size = 13) +
  theme(
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )

In [ ]:
ggsave('./figures_1/embryonic_stages_assignment_confidence_score_percell.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Anatomy

In [ ]:
length(unique(obj@meta.data$lv_anatomy_score))

round(length(unique(obj@meta.data$lv_anatomy_score))/100, 0)

In [ ]:
# Calculate 75th percentile
percentile_75 <- quantile(obj@meta.data$lv_anatomy_score, probs = 0.75, na.rm = TRUE)

# Compute max bin count for label positioning
hist_data <- hist(obj@meta.data$lv_anatomy_score, breaks = 1000, plot = FALSE)
max_y <- max(hist_data$counts)

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(data = obj@meta.data, aes(x = lv_anatomy_score)) +
  geom_histogram(
    bins = 1000, 
    color = 'blue', 
    fill = 'steelblue', 
    alpha = 0.85
  ) +
  geom_vline(xintercept = percentile_75, linetype = "dashed", color = "red", size = 0.5) +
  annotate(
    "text", 
    x = percentile_75, 
    y = max_y * 0.95, 
    label = "75th percentile", 
    size = 4.5, 
    color = "red", 
    fontface = "bold",
    hjust = -0.1
  ) +
  labs(
    title = "Anatomy",
    x = "Per-Cell Assignment Confidence Score (0 = low, 1 = high)",
    y = "Number of Cells"
  ) +
  theme_classic(base_size = 13) +
  theme(
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )

In [ ]:
ggsave('./figures_1/embryonic_anatomy_assignment_confidence_score_percell.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Somite Count

In [ ]:
length(unique(obj@meta.data$lv_somite_score))

round(length(unique(obj@meta.data$lv_somite_score))/100, 0)

In [ ]:
# Calculate 75th percentile
percentile_75 <- quantile(obj@meta.data$lv_somite_score, probs = 0.75, na.rm = TRUE)

# Compute max bin count for label positioning
hist_data <- hist(obj@meta.data$lv_somite_score, breaks = 1000, plot = FALSE)
max_y <- max(hist_data$counts)

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(data = obj@meta.data, aes(x = lv_somite_score)) +
  geom_histogram(
    bins = 1000, 
    color = 'blue', 
    fill = 'steelblue', 
    alpha = 0.85
  ) +
  geom_vline(xintercept = percentile_75, linetype = "dashed", color = "red", size = 0.5) +
  annotate(
    "text", 
    x = percentile_75, 
    y = max_y * 0.95, 
    label = "75th percentile", 
    size = 4.5, 
    color = "red", 
    fontface = "bold",
    hjust = -0.1
  ) +
  labs(
    title = "Somite Count",
    x = "Per-Cell Assignment Confidence Score (0 = low, 1 = high)",
    y = "Number of Cells"
  ) +
  theme_classic(base_size = 13) +
  theme(
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )

In [ ]:
ggsave('./figures_1/embryonic_somite_count_assignment_confidence_score_percell.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Cell Type

In [ ]:
length(unique(obj@meta.data$lv_cell_type_score))

round(length(unique(obj@meta.data$lv_cell_type_score))/100, 0)

In [ ]:
# Calculate 75th percentile
percentile_75 <- quantile(obj@meta.data$lv_cell_type_score, probs = 0.75, na.rm = TRUE)

# Compute max bin count for label positioning
hist_data <- hist(obj@meta.data$lv_cell_type_score, breaks = 1000, plot = FALSE)
max_y <- max(hist_data$counts)

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(data = obj@meta.data, aes(x = lv_cell_type_score)) +
  geom_histogram(
    bins = 1000, 
    color = 'blue', 
    fill = 'steelblue', 
    alpha = 0.85
  ) +
  geom_vline(xintercept = percentile_75, linetype = "dashed", color = "red", size = 0.5) +
  annotate(
    "text", 
    x = percentile_75, 
    y = max_y * 0.95, 
    label = "75th percentile", 
    size = 4.5, 
    color = "red", 
    fontface = "bold",
    hjust = 1.1
  ) +
  labs(
    title = "Cell Type",
    x = "Per-Cell Assignment Confidence Score (0 = low, 1 = high)",
    y = "Number of Cells"
  ) +
  theme_classic(base_size = 13) +
  theme(
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )

In [ ]:
ggsave('./figures_1/embryonic_extendend_cell_type_assignment_confidence_score_percell.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Cells

In [ ]:
length(unique(obj@meta.data$lv_cell_to_cell_score))

round(length(unique(obj@meta.data$lv_cell_to_cell_score))/100, 0)

In [ ]:
# Calculate 75th percentile
percentile_75 <- quantile(obj@meta.data$lv_cell_to_cell_score, probs = 0.75, na.rm = TRUE)

# Compute max bin count for label positioning
hist_data <- hist(obj@meta.data$lv_cell_to_cell_score, breaks = 1000, plot = FALSE)
max_y <- max(hist_data$counts)

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(data = obj@meta.data, aes(x = lv_cell_to_cell_score)) +
  geom_histogram(
    bins = 1000, 
    color = 'blue', 
    fill = 'steelblue', 
    alpha = 0.85
  ) +
  geom_vline(xintercept = percentile_75, linetype = "dashed", color = "red", size = 0.5) +
  annotate(
    "text", 
    x = percentile_75, 
    y = max_y * 0.95, 
    label = "75th percentile", 
    size = 4.5, 
    color = "red", 
    fontface = "bold",
    hjust = 1.1
  ) +
  labs(
    title = "Cell to Cell Match",
    x = "Per-Cell Assignment Confidence Score (0 = low, 1 = high)",
    y = "Number of Cells"
  ) +
  theme_classic(base_size = 13) +
  theme(
    axis.text = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )

In [ ]:
ggsave('./figures_1/embryonic_extendend_cell_type_assignment_confidence_score_percell.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

# BarPlots

## Embryonic Stages

In [ ]:
table(obj@meta.data$lv_stages)

In [ ]:
# Prepare data frame for plotting (all stages, original names)
stage_counts <- obj@meta.data %>%
  group_by(lv_stages) %>%
  summarise(Cell_Count = n()) %>%
  mutate(lv_stages = factor(
    lv_stages,
    levels = c(
      "E6.5", "E7.0", "E7.25", "E7.5", "E7.75", "E8.0", "E8.25", "E8.5",
      "E8.75", "E9.0", "E9.25", "E9.5", "Mixed gastrulation"
    )
  ))

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(stage_counts, aes(x = lv_stages, y = Cell_Count)) +
  geom_bar(stat = "identity", fill = "steelblue", color = "blue", width = 0.8) +
  labs(
    title = "Number of Cells per Embryonic Stage",
    x = "Embryonic Stage",
    y = "Number of Cells"
  ) +
  scale_x_discrete(labels = c(
    "E6.5" = "E6.5", "E7.0" = "E7.0", "E7.25" = "E7.25", "E7.5" = "E7.5",
    "E7.75" = "E7.75", "E8.0" = "E8.0", "E8.25" = "E8.25", "E8.5" = "E8.5",
    "E8.75" = "E8.75", "E9.0" = "E9.0", "E9.25" = "E9.25", "E9.5" = "E9.5",
    "Mixed gastrulation" = "Mixed"
  )) +
  theme_classic(base_size = 13) +
  theme(
    axis.text.x = element_text(size = 14, angle = 35, hjust = 1, vjust = 1),
    axis.text.y = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )


In [ ]:
ggsave('./figures_1/nof_cells_embryonic_stages.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Anatomy

In [ ]:
table(obj@meta.data$lv_anatomy)

In [ ]:
# Prepare data for plotting (keep original names in factor order)
anatomy_levels <- c(
  "Pooled", "YS", "EP", "Anterior", "Posterior",
  "Anterior section", "Medial section", "Posterior section"
)

anatomy_counts <- obj@meta.data %>%
  group_by(lv_anatomy) %>%
  summarise(Cell_Count = n()) %>%
  mutate(lv_anatomy = factor(lv_anatomy, levels = anatomy_levels))

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(anatomy_counts, aes(x = lv_anatomy, y = Cell_Count)) +
  geom_bar(stat = "identity", fill = "steelblue", color = "blue", width = 0.8) +
  labs(
    title = "Number of Cells per Anatomical Region",
    x = "Anatomical Region",
    y = "Number of Cells"
  ) +
  scale_x_discrete(labels = c(
    "Pooled" = "Pooled",
    "YS" = "YS",
    "EP" = "EP",
    "Anterior" = "Anterior",
    "Posterior" = "Posterior",
    "Anterior section" = "Ant. sec.",
    "Medial section" = "Med. sec.",
    "Posterior section" = "Post. sec."
  )) +
  theme_classic(base_size = 13) +
  theme(
    axis.text.x = element_text(size = 14, angle = 35, hjust = 1, vjust = 1),
    axis.text.y = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )


In [ ]:
ggsave('./figures_1/nof_cells_anatomy.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Somite Count

In [ ]:
table(obj@meta.data$lv_somite)

In [ ]:
# Define desired factor order (numbers as strings, Pooled last)
somite_levels <- c(
  "6", "7", "8", "10", "12", "14", "15", "16", "18",
  "20", "21", "22", "23", "24", "Pooled"
)

# Prepare data for plotting
somite_counts <- obj@meta.data %>%
  group_by(lv_somite) %>%
  summarise(Cell_Count = n()) %>%
  mutate(lv_somite = factor(lv_somite, levels = somite_levels))

options(repr.plot.width = 6.4, repr.plot.height = 5)

ggplot(somite_counts, aes(x = lv_somite, y = Cell_Count)) +
  geom_bar(stat = "identity", fill = "steelblue", color = "blue", width = 0.8) +
  labs(
    title = "Number of Cells per Somite Count",
    x = "Somite Count",
    y = "Number of Cells"
  ) +
  scale_x_discrete(labels = c(
    setNames(somite_levels, somite_levels)[1:14], # numbers remain unchanged
    "Pooled" = "Pool."
  )) +
  theme_classic(base_size = 13) +
  theme(
    axis.text.x = element_text(size = 14, angle = 35, hjust = 1, vjust = 1),
    axis.text.y = element_text(size = 14),
    axis.title = element_text(size = 14, face = "bold"),
    plot.title = element_text(size = 15, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )


In [ ]:
ggsave('./figures_1/nof_cells_somite_count.pdf', plot = last_plot(), width = 6.4, height = 5, units = 'in', dpi = 300)

## Cell Type

In [ ]:
table(obj@meta.data$lv_cell_type)

In [ ]:
# Calculate cell counts per type
celltype_counts <- obj@meta.data %>%
  group_by(lv_cell_type) %>%
  summarise(Cell_Count = n()) %>%
  arrange(desc(Cell_Count)) # Optional: order bars by count

# Abbreviate long labels for plotting (add more as needed)
celltype_labels <- unique(obj@meta.data$lv_cell_type)

# Set default: show all labels, but overwrite selected
all_labels <- setNames(as.character(celltype_counts$lv_cell_type), celltype_counts$lv_cell_type)
all_labels[names(celltype_labels)] <- celltype_labels

options(repr.plot.width = 12.8, repr.plot.height = 7.5)

ggplot(celltype_counts, aes(x = lv_cell_type, y = Cell_Count)) +
  geom_bar(stat = "identity", fill = "steelblue", color = "blue", width = 0.8) +
  labs(
    title = "Number of Cells per Cell Type",
    x = "Cell Type",
    y = "Number of Cells"
  ) +
  scale_x_discrete(labels = all_labels) +
  theme_classic(base_size = 13) +
  theme(
    axis.text.x = element_text(size = 13, angle = 65, hjust = 1, vjust = 1),
    axis.text.y = element_text(size = 13),
    axis.title = element_text(size = 13, face = "bold"),
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5, margin = margin(b = 12)),
    panel.grid.major.y = element_line(color = "grey90", size = 0.4),
    axis.line = element_line(color = "black", size = 0.5),
    plot.margin = margin(10, 10, 10, 10)
  )


In [ ]:
ggsave('./figures_1/nof_cell_type_count.pdf', plot = last_plot(), width = 12.8, height = 7.5, units = 'in', dpi = 300)

### Multi parameters for cell type

In [ ]:
df <- obj@meta.data[,c('v4k_pca50_predicted.id','v4k_pca75_predicted.id','v4k_pca100_predicted.id',
 'v7k_pca50_predicted.id','v7k_pca75_predicted.id','v7k_pca100_predicted.id',
 'v10k_pca50_predicted.id','v10k_pca75_predicted.id','v10k_pca100_predicted.id')]

In [ ]:
head(df)

In [ ]:
colnames(df) <- c('v4k_pca50','v4k_pca75','v4k_pca100',
 'v7k_pca50','v7k_pca75','v7k_pca100',
 'v10k_pca50','v10k_pca75','v10k_pca100')

In [ ]:
head(df)

In [ ]:
library(tidyr)

In [ ]:

df_long <- df %>%
  mutate(cell = rownames(df)) %>%
  pivot_longer(
    cols = -cell,
    names_to = "prediction_method",
    values_to = "cell_type"
  )

cell_counts <- df_long %>%
  group_by(prediction_method, cell_type) %>%
  summarise(cell_number = n(), .groups = "drop")

# Calculate frequency per prediction method
cell_freq <- cell_counts %>%
  group_by(prediction_method) %>%
  mutate(frequency = cell_number / sum(cell_number)) %>%
  ungroup()

# Normalize frequency within each cell_type (row)
cell_freq <- cell_freq %>%
  group_by(cell_type) %>%
  mutate(row_scaled_frequency = frequency / max(frequency, na.rm = TRUE)) %>%
  ungroup()


In [ ]:
head(cell_counts)

In [ ]:

# Order for plotting
cell_types_order <- cell_freq %>%
  group_by(cell_type) %>%
  summarise(total = sum(frequency)) %>%
  arrange(desc(total)) %>%
  pull(cell_type)

methods_order <- unique(cell_freq$prediction_method)

options(repr.plot.width = 20, repr.plot.height = 20)

ggplot(cell_freq, aes(
  x = factor(prediction_method, levels = methods_order),
  y = factor(cell_type, levels = cell_types_order),
  fill = row_scaled_frequency
)) +
  geom_tile(color = "grey90", size = 0.4) +  # Subtle tile borders
  scale_fill_viridis_c(
    option = "viridis",
    name = "Row-Scaled Frequency",
    limits = c(0, 1),
    guide = guide_colorbar(
      barwidth = 1.5, barheight = 15, frame.colour = "black", frame.linewidth = 0.6
    )
  ) +
  labs(
    x = "Prediction Parameters",
    y = "Cell Type",
    title = "Row-Normalized Cell Type Frequency per Prediction Parameter"
  ) +
  theme_minimal(base_size = 18) +
  theme(
    plot.title = element_text(size = 28, face = "bold", hjust = 0.5, margin = margin(b = 16)),
    axis.title.x = element_text(size = 22, face = "bold", margin = margin(t = 14)),
    axis.title.y = element_text(size = 22, face = "bold", margin = margin(r = 14)),
    axis.text.x = element_text(size = 15, angle = 35, hjust = 1, vjust = 1, face = "bold"),
    axis.text.y = element_text(size = 16, face = "bold"),
    panel.grid = element_blank(),
    plot.margin = margin(20, 20, 20, 20),
    legend.title = element_text(size = 18, face = "bold"),
    legend.text = element_text(size = 16),
    panel.border = element_rect(color = "black", fill = NA, linewidth = 1.1)
  )



In [ ]:
ggsave('./figures_1/nof_cell_type_count_methods.pdf', plot = last_plot(), width = 20, height = 20, units = 'in', dpi = 300)

In [ ]:
# cell_counts: number of cells per prediction method and cell type
cell_counts <- df_long %>%
  group_by(prediction_method, cell_type) %>%
  summarise(cell_number = n(), .groups = "drop") %>%
  arrange(prediction_method, desc(cell_number))

# View as a table
head(cell_counts, 20)   # View first 20 rows


In [ ]:
# Calculate frequency per prediction_method
cell_freq <- cell_counts %>%
  group_by(prediction_method) %>%
  mutate(frequency = cell_number / sum(cell_number)) %>%
  ungroup() %>%
  arrange(prediction_method, desc(frequency)) %>%
  mutate(
    percent = sprintf("%.2f%%", frequency * 100)  # Format as percent string
  )

# View as a table
head(cell_freq, 20)   # View first 20 rows


In [ ]:

# Using cell_freq as example (works identically for cell_counts)
cell_freq <- cell_freq %>%
  separate(
    col = prediction_method,
    into = c("Variable_Genes_Count", "PCA_Count", "rest"),
    sep = "_",
    remove = FALSE
  ) %>%
  # Remove "rest" (if present) and clean up columns
  select(-rest) %>%
  # Optionally, remove "v" and "k" from gene count, "pca" from PCA count for readability
  mutate(
    Variable_Genes_Count = gsub("v|k", "", Variable_Genes_Count),
    Variable_Genes_Count = paste0(Variable_Genes_Count, "K"),
    PCA_Count = gsub("pca", "", PCA_Count)
  )

# Preview the result
head(cell_freq, 10)

In [ ]:
tail(cell_freq, 10)

In [ ]:
write.table(cell_freq, "./figures_1/mapping_methods_1.txt",quote = FALSE, sep = '\t',col.names = TRUE, dec = ',')
write.table(cell_freq, "./figures_1/mapping_methods_2.txt",quote = FALSE, sep = '\t',col.names = TRUE, dec = '.')

# UMAP

In [ ]:
atlas

In [ ]:
obj

In [ ]:
# Plot

In [ ]:
mapped_cells <- unique(obj$lv_cell_to)
length(mapped_cells)

In [ ]:
DimPlot(obj, reduction = 'UMAP', group.by = '')

In [ ]:
obj$v7k

In [ ]:
options(repr.plot.width = 12.8, repr.plot.height = 10)

p <- DimPlot(
  object          = atlas,
  reduction       = 'UMAP',
  cells.highlight = mapped_cells,
  cols.highlight  = "firebrick2",     # a bit brighter
  cols            = "grey90",         # softer background
  pt.size         = 0.15,             # slightly larger points
  sizes.highlight = 2.5,              # bigger highlight
  alpha           = 0.7,              # more visible highlight
  raster          = FALSE
) +
  NoAxes() +
  NoLegend() +
  ggtitle("Atlas Mapping: Used Reference Cells") +
  theme(
    plot.title = element_text(
      size = 28, face = "bold", hjust = 0.5, margin = margin(b = 15)
    ),
    plot.margin = margin(20, 20, 20, 20)
  )

p


In [ ]:
ggsave('./figures_1/atlas_umap_reference_cells.pdf', plot = last_plot(), width = 12.8, height = 10, units = 'in', dpi = 300)
ggsave('./figures_1/atlas_umap_reference_cells.png', plot = last_plot(), width = 12.8, height = 10, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)

p <- DimPlot(
  object          = atlas,
  reduction       = 'UMAP',
  cells.highlight = mapped_cells,
  cols.highlight  = "firebrick2",     # a bit brighter
  cols            = "grey90",         # softer background
  pt.size         = 0.15,             # slightly larger points
  sizes.highlight = 2.5,              # bigger highlight
  alpha           = 0.7,              # more visible highlight
  raster          = FALSE
) +
  NoAxes() +
  NoLegend() +
  ggtitle("Atlas Mapping: Used Reference Cells") +
  theme(
    plot.title = element_text(
      size = 28, face = "bold", hjust = 0.5, margin = margin(b = 15)
    ),
    plot.margin = margin(20, 20, 20, 20)
  )

p


In [ ]:
ggsave('./figures_1/atlas_umap_reference_cells_2.pdf', plot = last_plot(), width = 10, height = 10, units = 'in', dpi = 300)
ggsave('./figures_1/atlas_umap_reference_cells_2.png', plot = last_plot(), width = 10, height = 10, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 9.6, repr.plot.height = 6)
DimPlot(obj, reduction = "UMAP", group.by = 'lv_anatomy',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())


In [ ]:
ggsave('./figures_1/obj_anatomy.pdf', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)
ggsave('./figures_1/obj_anatomy.png', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)

In [ ]:
# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$lv_anatomy)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 1]
length(celltypes_gt10)
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = lv_anatomy == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 6 )
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 2)
combined_plot

In [ ]:
ggsave('./figures_1/obj_lv_anatomy_greater_1.pdf', plot = last_plot(), width = 20, height = 6, units = 'in', dpi = 300)
ggsave('./figures_1/obj_lv_anatomy_greater_1.png', plot = last_plot(), width = 20, height = 6, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 9.6, repr.plot.height = 6)
DimPlot(obj, reduction = "UMAP", group.by = 'lv_somite',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())


In [ ]:
ggsave('./figures_1/obj_somite_count.pdf', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)
ggsave('./figures_1/obj_somite_count.png', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)

In [ ]:
# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$lv_somite)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 1]
length(celltypes_gt10)
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = lv_somite == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 9 )
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 3)
combined_plot

In [ ]:
ggsave('./figures_1/obj_lv_somite_greater_1.pdf', plot = last_plot(), width = 20, height = 9, units = 'in', dpi = 300)
ggsave('./figures_1/obj_lv_somite_greater_1.png', plot = last_plot(), width = 20, height = 9, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 9.6, repr.plot.height = 6)
DimPlot(obj, reduction = "UMAP", group.by = 'lv_stages',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())


In [ ]:
ggsave('./figures_1/obj_embryonic_stages.pdf', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)
ggsave('./figures_1/obj_embryonic_stages.png', plot = last_plot(), width = 9.6, height = 6, units = 'in', dpi = 300)

In [ ]:
# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$lv_stages)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 1]
length(celltypes_gt10)
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = lv_stages == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 9 )
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 3)
combined_plot

In [ ]:
ggsave('./figures_1/obj_lv_embryonic_stages_greater_1.pdf', plot = last_plot(), width = 20, height = 9, units = 'in', dpi = 300)
ggsave('./figures_1/obj_lv_embryonic_stages_1.png', plot = last_plot(), width = 20, height = 9, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
DimPlot(obj, reduction = "UMAP", group.by = 'lv_cell_type',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())

In [ ]:
ggsave('./figures_1/obj_celltypes_v7k75pca.pdf', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v7k75pca.png', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)

In [ ]:

# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$lv_cell_type)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 6]
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = lv_cell_type == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
library(patchwork)

In [ ]:
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 8)
combined_plot

In [ ]:
ggsave('./figures_1/obj_celltypes_v7k75pca_umap_greater_6.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v7k75pca_umap_greater_6.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
DimPlot(obj, reduction = "UMAP", group.by = 'v4k_pca50_predicted.id',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())

In [ ]:
ggsave('./figures_1/obj_celltypes_v4k50pca.pdf', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v4k50pca.png', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)

In [ ]:
# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$v4k_pca50_predicted.id)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 6]
length(celltypes_gt10)
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = v4k_pca50_predicted.id == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 8)
combined_plot

In [ ]:
ggsave('./figures_1/obj_celltypes_v4k50pca_umap_greater_6.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v4k50pca_umap_greater_6.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
DimPlot(obj, reduction = "UMAP", group.by = 'v4k_pca50_predicted.id',cols = r3dcol$cols_94, alpha = 0.5)+ NoAxes() + theme(plot.title = element_blank())

In [ ]:
ggsave('./figures_1/obj_celltypes_v4k50pca.pdf', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v4k50pca.png', plot = last_plot(), width = 20, height = 8, units = 'in', dpi = 300)

In [ ]:
# Get the 40 cell types you want to highlight
celltype_counts <- table(obj$v10k_pca100_predicted.id)
celltypes_gt10 <- names(celltype_counts)[celltype_counts > 6]
length(celltypes_gt10)
celltypes_gt10 <- head(celltypes_gt10, 40)  # in case you want only the top 40

# Get cell names for each type
cells_by_type <- lapply(celltypes_gt10, function(ct) {
  WhichCells(obj, expression = v10k_pca100_predicted.id == ct)
})
names(cells_by_type) <- celltypes_gt10

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
plot_list <- lapply(seq_along(cells_by_type), function(i) {
  ct <- names(cells_by_type)[i]
  cells <- cells_by_type[[i]]
  
  DimPlot(
    obj,
    reduction = "UMAP",
    cells.highlight = list(cells),
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.5,
    sizes.highlight = 2.5,
    alpha = 0.5,
      raster = TRUE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle(ct) +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )
})


In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
combined_plot <- wrap_plots(plot_list, ncol = 5, nrow = 8)
combined_plot

In [ ]:
ggsave('./figures_1/obj_celltypes_v10k100pca_umap_greater_6.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/obj_celltypes_v10k100pca_umap_greater_6.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

# Save

In [ ]:
saveRDS(obj, file = '../../cmo_version3.rds', compress = FALSE)

# Dynamics

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p1 <- VlnPlot(obj, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'lv_stages', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())

p1

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p2 <- VlnPlot(obj, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'lv_somite', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())
p2

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p3 <- VlnPlot(obj, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'lv_anatomy', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())
p3

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p1a <- VlnPlot(atlas, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'stage', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())

p1a

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p2a <- VlnPlot(atlas, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'somite_count', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())

p2a

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p3a <- VlnPlot(atlas, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'anatomy', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())

p3a

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
p4a <- VlnPlot(atlas, features = c("Atf3", "Zfp711", "Bcl6b"), group.by = 'celltype_extended_atlas', cols = r3dcol$cols_94, log = FALSE,alpha = 0.5, raster = TRUE) & 
scale_x_discrete(labels = c("Mixed gastrulation" = "Mixed"))&
theme(axis.title.x =  element_blank())

p4a

In [ ]:
library(ComplexHeatmap)

In [ ]:
library(pheatmap)

In [ ]:
ko_genes <- c('Atf3','Zfp711','Bcl6b')

In [ ]:
avg_ko <- AverageExpression(obj,features = ko_genes, layer = 'counts',group.by =  'cell_type_subclusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Average Expression on Subclusters" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.5, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_obj_subclusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
avg_ko <- AverageExpression(obj,features = ko_genes, layer = 'counts',group.by =  'cell_type_clusters' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Average Expression on Clusters" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.5, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_obj_clusters.pdf",width = 6.5, height = 4)
print(x)
dev.off()   


In [ ]:
#atlas$celltype_extended_atlas

In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'celltype_extended_atlas' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Average Expression on Extended Mouse Gastrulation Atlas - Cell Type" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_atlas.pdf",width = 14, height = 5)
print(x)
dev.off()   


In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'stage' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Embryonic Stage" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_atlas_embryonic_stage.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'somite_count' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Somite Count" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_atlas_somite_count.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:
avg_ko <- AverageExpression(atlas,features = ko_genes, layer = 'counts',group.by =  'anatomy' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$originalexp)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "Anatomy" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_atlas_anatomy.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


# scMULTIOME

In [ ]:
mutiome <- readRDS('/home/ridvan/PhD_Projects/Multiome/000_io/outputs/multiome_only_2.rds')

In [ ]:
mutiome

In [ ]:
table(mutiome$orig.ident)

In [ ]:
DefaultAssay(mutiome) <- "RNA"

In [ ]:
avg_ko <- AverageExpression(mutiome,features = ko_genes, layer = 'counts',group.by =  'orig.ident' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In Vitro Differentiation Days" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 6.4, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_multiome_days.pdf",width = 6.4, height = 5)
print(x)
dev.off()   


In [ ]:
table(mutiome$annotation_medium)

In [ ]:
avg_ko <- AverageExpression(mutiome,features = ko_genes, layer = 'counts',group.by =  'annotation_medium' )

In [ ]:
avg_ko

In [ ]:
dense_matrix<- as.matrix(avg_ko$RNA)

In [ ]:
dense_matrix

In [ ]:
x <- pheatmap(
  dense_matrix,
  scale = "row",                        # Scale rows for normalization
  clustering_distance_rows = "euclidean", # Cluster rows by Euclidean distance
  cluster_rows = FALSE,                  # Enable row clustering
  cluster_cols = FALSE,                 # Preserve column order
  show_rownames = TRUE,                 # Display row names
  show_colnames = TRUE,                 # Display column names
  fontsize_row = 10,                     # Font size for row labels
  fontsize_col = 10,                     # Font size for column labels
  color = colorRampPalette(c("blue", "white", "red"))(100),                 # Use viridis color palette
  border_color = "lightgrey",                    # Remove cell borders for a cleaner look
  cellwidth = 10,                       # Adjust cell width
  cellheight = 12,                      # Adjust cell height
  main = "In Vitro Differentiation Cell Types" ,  # Add a title
)

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5 )
print(x)

In [ ]:

pdf("./figures_1/average_expression_multiome_cell_types.pdf",width = 9, height = 5)
print(x)
dev.off()   


In [ ]:
mutiome$8k_direct_pca_2_predicted.id

In [ ]:
library(Signac)

In [ ]:
DefaultAssay(mutiome) <- 'ATAC'

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20 )
CoveragePlot(mutiome, region = 'Atf3', features = 'Atf3', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_clusters_atf3.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_clusters_atf3.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$orig.ident

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
CoveragePlot(mutiome, region = 'Atf3', features = 'Atf3', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_days_atf3.pdf', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_days_atf3.png', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$`8k_direct_pca_2_predicted.id`

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
CoveragePlot(mutiome, region = 'Atf3', features = 'Atf3', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_8k_direct_pca_2_atf3.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_8k_direct_pca_2_atf3.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
# ------------------------------------------------


In [ ]:
Idents(mutiome) <- mutiome$annotation_medium

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20 )
CoveragePlot(mutiome, region = 'Zfp711', features = 'Zfp711', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_clusters_zfp711.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_clusters_zfp711.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$orig.ident

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
CoveragePlot(mutiome, region = 'Zfp711', features = 'Zfp711', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_days_zfp711.pdf', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_days_zfp711.png', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$`8k_direct_pca_2_predicted.id`

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
CoveragePlot(mutiome, region = 'Zfp711', features = 'Zfp711', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_8k_direct_pca_2_Zfp711.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_8k_direct_pca_2_Zfp711.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$annotation_medium

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20 )
CoveragePlot(mutiome, region = 'Bcl6b', features = 'Bcl6b', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_clusters_bcl6b.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_clusters_bcl6b.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$orig.ident

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 5 )
CoveragePlot(mutiome, region = 'Bcl6b', features = 'Bcl6b', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_days_bcl6b.pdf', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_days_bcl6b.png', plot = last_plot(), width = 20, height = 5, units = 'in', dpi = 300)

In [ ]:
Idents(mutiome) <- mutiome$`8k_direct_pca_2_predicted.id`

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 22.5 )
CoveragePlot(mutiome, region = 'Bcl6b', features = 'Bcl6b', assay = 'ATAC',extend.upstream = 10000, extend.downstream = 10000, expression.assay = 'RNA', peaks = TRUE)

In [ ]:
ggsave('./figures_1/mutiome_8k_direct_pca_2_Bcl6b.pdf', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)
ggsave('./figures_1/mutiome_8k_direct_pca_2_Bcl6b.png', plot = last_plot(), width = 20, height = 22.5, units = 'in', dpi = 300)

# CD45

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 18 )
DotPlot(atlas, features = c("Ptprc",'Itga2b',"Fcgr3","Fcgr2b", "Kit"), group.by = "celltype_extended_atlas")

In [ ]:
ggsave('./figures_1/cd45_atlas_dot.pdf', plot = last_plot(), width = 9, height = 18, units = 'in', dpi = 300)
ggsave('./figures_1/cd45_atlas_dot.png', plot = last_plot(), width = 9, height = 18, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5 )
DotPlot(obj, features = c("Ptprc",'Itga2b',"Fcgr3","Fcgr2b", "Kit"), group.by = "cell_type_clusters")

In [ ]:
ggsave('./figures_1/cd45_cmo_dot.pdf', plot = last_plot(), width = 9, height = 18, units = 'in', dpi = 300)
ggsave('./figures_1/cd45_cmo_dot.png', plot = last_plot(), width = 9, height = 18, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 6 )
FeaturePlot(obj, features = "Ptprc", order = TRUE,reduction = "UMAP")+ NoAxes()x

In [ ]:
ggsave('./figures_1/cd45_cmo_feature.pdf', plot = last_plot(), width = 9, height = 6, units = 'in', dpi = 300)
ggsave('./figures_1/cd45_cmo_feature.png', plot = last_plot(), width = 9, height = 6, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8 )
FeaturePlot(atlas, features = "Ptprc", order = TRUE,reduction = "UMAP", raster = FALSE,    pt.size = 0.2)+ NoAxes()

In [ ]:
ggsave('./figures_1/cd45_atlas_feature.pdf', plot = last_plot(), width = 8, height = 8, units = 'in', dpi = 300)
ggsave('./figures_1/cd45_atlas_feature.png', plot = last_plot(), width = 8, height = 8, units = 'in', dpi = 300)

In [ ]:
cells <- WhichCells(atlas, expression = celltype_extended_atlas == "EMP")
length(cells)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8 )
  DimPlot(
    atlas,
    reduction = "UMAP",
    cells.highlight = cells,
    cols.highlight = "firebrick",
    cols = "grey90",
    pt.size = 0.2,
    sizes.highlight = 0.2,
    alpha = 0.5,
      raster = FALSE
  ) +
    NoAxes() +
    NoLegend() +
    ggtitle("EMP") +
    theme(
      plot.title = element_text(size = 13,  hjust = 0.5),
      plot.margin = margin(2, 2, 2, 2)
    )

In [ ]:
ggsave('./figures_1/cd45_emp_atlas_dimplot_highlight.pdf', plot = last_plot(), width = 8, height = 8, units = 'in', dpi = 300)
ggsave('./figures_1/cd45_emp_atlas_dimplot_highlight.png', plot = last_plot(), width = 8, height = 8, units = 'in', dpi = 300)

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)
DimPlot(obj, reduction = 'UMAP', group.by = 'cell_type_subclusters', label = T,label.size = 5, alpha=0.7, cols = rev(r3dcol$cols_46), repel= TRUE, label.box = FALSE )

In [ ]:
ggsave('./figures_1/obj_umap_sub.pdf', plot = last_plot(), width = 20, height = 10, units = 'in', dpi = 300)
ggsave('./figures_1/obj_umap_sub.png', plot = last_plot(), width = 20, height = 10, units = 'in', dpi = 300)
